In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()

import torch 
import torch.nn as nn 
import torch.optim as optim


from models.learned_qp_solver_bi_cycle_steer_part import MLP_Init as MLP_Init_steer
from models.learned_qp_solver_bi_cycle_steer_part import learned_qp_solver as learned_qp_solver_steer


from models.learned_qp_solver_bi_cycle_vel_part import MLP_Init as MLP_Init_vel
from models.learned_qp_solver_bi_cycle_vel_part import learned_qp_solver as learned_qp_solver_vel

from scipy.io import loadmat
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
num = 50
t = 0.1


vel_min = 0.0
vel_max = 30.0

acc_min = -6.0
acc_max = 6.0

jerk_min = -6.0
jerk_max = 6.0

steer_min = -0.8
steer_max = 0.8

steerdot_min = -1.0
steerdot_max = 1.0

steerddot_min = -1.0
steerddot_max = 1.0

In [ ]:
v_des =  np.random.uniform(vel_min, vel_max)

num_batch = 500

# vel_samples = np.random.uniform(vel_min, vel_max,  (num_batch, num)    )
vel_samples = np.random.multivariate_normal(v_des*np.ones(num), 10*np.identity(num), (num_batch, ) )


vel_samples = np.clip( vel_samples, vel_min, vel_max  )


# vel_init = np.random.uniform(vel_min, vel_max)*np.ones((num_batch , 1)) 
vel_init = v_des*np.ones((num_batch , 1)) 


inp_vel = np.hstack(( vel_samples, vel_init))

median_vel = np.array([14.99712617, 14.98165856, 15.00156322, 15.0315946 , 15.0237649 ,
       14.95290831, 15.02362848, 15.00488991, 14.98256721, 15.02511973,
       15.01874045, 15.01791766, 14.94479826, 15.02234513, 15.00146979,
       15.00260772, 15.04212057, 14.98586476, 14.97039095, 15.01680088,
       15.02091521, 14.99245809, 15.02324861, 14.97728478, 14.98595452,
       15.00038151, 14.96556379, 14.98516695, 15.02776369, 15.03157252,
       15.01656847, 15.04913992, 15.032816  , 15.02177236, 14.95636724,
       15.00040805, 15.00483608, 15.00281992, 14.98058886, 15.04900013,
       15.02005837, 15.00121925, 14.97157685, 15.00349338, 14.92630127,
       14.99345513, 15.02403063, 14.98243826, 14.99892036, 15.03160816,
       15.00260323])
iqr_vel = np.array([15.00489034, 14.96979553, 15.02179493, 14.98077341, 14.9401202 ,
       14.94736427, 15.01472935, 14.98427932, 15.02928112, 14.96119056,
       14.98033242, 15.00625851, 14.88887973, 15.00977705, 14.98675345,
       15.01435311, 14.95118661, 15.05150952, 15.03881341, 15.03805316,
       14.99600287, 14.96569228, 14.93651456, 15.06040163, 14.98350009,
       15.02995742, 14.91813083, 15.00162581, 14.98425137, 15.07897904,
       14.94457048, 14.97445598, 14.98252818, 14.99157274, 14.98652712,
       14.99993726, 15.03111512, 15.01579364, 14.96811747, 14.98984632,
       15.02163057, 15.01268755, 15.03481071, 14.99005507, 15.01660515,
       15.03712912, 14.95232336, 15.01972352, 15.03382282, 14.98628055,
       14.99610719])



median_vel = torch.tensor(median_vel).float().to(device)
iqr_vel = torch.tensor(iqr_vel).float().to(device)

In [ ]:
steer_des =  np.random.uniform(steer_min, steer_max)

num_batch = 500

# vel_samples = np.random.uniform(vel_min, vel_max,  (num_batch, num)    )
steer_samples = np.random.multivariate_normal(steer_des*np.ones(num), 1*np.identity(num), (num_batch, ) )


steer_samples = np.clip( steer_samples, steer_min, steer_max  )


# vel_init = np.random.uniform(vel_min, vel_max)*np.ones((num_batch , 1)) 
steer_init = steer_des*np.ones((num_batch , 1)) 


inp_steer = np.hstack(( steer_samples, steer_init))


median_steer = np.array([-4.30842576e-04,  5.91636819e-05, -1.98533905e-03,  1.95326047e-03,
       -1.39358004e-03, -5.68464714e-05,  2.65373538e-04, -2.05526464e-03,
       -7.79943244e-04, -2.89536127e-04, -1.67782600e-03, -1.30369985e-03,
        1.54351594e-03, -2.10914668e-03, -8.94449061e-04, -1.12731106e-04,
        1.39067091e-03,  2.35456389e-03,  1.13105755e-03, -1.60172509e-03,
        5.96616154e-04,  4.50916455e-04,  1.03440312e-03,  2.20521562e-03,
        1.91066337e-03, -2.36909941e-03,  5.23557830e-04,  3.22519684e-03,
       -6.10364108e-04, -1.55881571e-03, -2.54650519e-04,  5.49939732e-04,
       -2.54372567e-03,  1.15624681e-03,  3.09751082e-03, -1.01707749e-03,
       -1.16847934e-03, -9.60229520e-04,  3.19715942e-04, -7.80995571e-04,
        3.91363172e-04,  1.96635947e-05, -2.01634184e-04, -1.34935042e-03,
       -1.25220145e-03,  1.28156314e-04,  1.63297072e-03, -2.31602556e-03,
        8.75998110e-04, -2.06147938e-04,  1.15305881e-03])
iqr_steer = np.array([0.80001846, 0.79929592, 0.79676276, 0.79730429, 0.79646984,
       0.79966968, 0.80083456, 0.80339261, 0.79957275, 0.80271643,
       0.79956587, 0.7986954 , 0.80184285, 0.79680465, 0.79700468,
       0.80111911, 0.80292797, 0.79760931, 0.8001852 , 0.79991013,
       0.80251082, 0.80016607, 0.80274812, 0.7996054 , 0.79708629,
       0.79890295, 0.8011802 , 0.79833857, 0.80142342, 0.80023931,
       0.80009408, 0.79791844, 0.79955642, 0.79824443, 0.79912222,
       0.79983322, 0.80107271, 0.79787763, 0.80142027, 0.79819341,
       0.79860864, 0.79933899, 0.7980339 , 0.79911936, 0.79988434,
       0.79932644, 0.79560559, 0.79765311, 0.79994117, 0.80107361,
       0.79731313])


median_steer = torch.tensor(median_steer).float().to(device)
iqr_steer = torch.tensor(iqr_steer).float().to(device)

In [ ]:
mlp_init_inp_dim_vel = np.shape(inp_vel)[1]
mlp_init_hidden_dim_vel = 1024
# mlp_init_out_dim = (2*nvar)**2+2*nvar+2*nvar
mlp_init_out_dim = num+(2*num+2*(num-1)+2*(num-2))

# mlp_pred = MLP_Pred(mlp_pred_inp_dim, mlp_pred_hidden_dim, mlp_pred_out_dim  )
mlp_init_vel = MLP_Init_vel(mlp_init_inp_dim_vel, mlp_init_hidden_dim_vel, mlp_init_out_dim  )


model_vel = learned_qp_solver_vel(num_batch, num, t, mlp_init_vel).to(device)
model_vel.load_state_dict(torch.load('./weights/learned_qp_bi_cycle_vel_part.pth'))


In [ ]:

mlp_init_inp_dim_steer = np.shape(inp_steer)[1]
mlp_init_hidden_dim_steer = 1024
# mlp_init_out_dim = (2*nvar)**2+2*nvar+2*nvar
mlp_init_out_dim_steer = num+(2*num+2*(num-1)+2*(num-2))
# print((2*num+2*(num-1)+2*(num-2)))

# mlp_pred = MLP_Pred(mlp_pred_inp_dim, mlp_pred_hidden_dim, mlp_pred_out_dim  )
mlp_init_steer = MLP_Init_steer(mlp_init_inp_dim_steer, mlp_init_hidden_dim_steer, mlp_init_out_dim_steer  )


model_steer = learned_qp_solver_steer(num_batch, num, t, mlp_init_steer).to(device)
model_steer.load_state_dict(torch.load('./weights/learned_qp_bi_cycle_steer_part.pth'))